# Исследование надежности заемщиков.

**Цель проекта:** на полученных из банка данных с характеристиками заёмщиков (пол, возраст и другие, суммарно около 10 характеристик) проверить, если ли зависимость между фактом непогашения кредита в срок и: 
 - семейным положением
 - количеством детей
 - уровнем дохода
 - целью кредита
 
Результаты исследования будут использованы для модификации модели кредитного скоринга.

## Обзор данных:

In [46]:
import pandas as pd

In [47]:
#загрузим данные и отобразим сводную информацию по таблице
try:
    df = pd.read_csv('datasets/data.csv')
    df.info()
except:
    display('Проверьте доступ к данным')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В полученной таблице 12 столбцов, более 21тыс. строк. Имеются пропуски и вещественный тип данных в двух столбцах. Это ненормально, будет нам мешать - исправим.

### Заполнение пропусков

Пропуски имеются в двух столбцах: `days_employed` и `total_income`. Причина - скорее всего нежелание клиента делиться информацией о стаже и доходах с банком.  
Пропущенные значения - количественные. Заполним их медианой - тогда крайние (минимальные и максимальные), нехарактерные для столбцов значения (из-за ошибок ввода или по другим причинам) не так сильно повлияют на статистику.

In [48]:
#отобразим количество строк с пропусками
none_rows = df[df.isna().any(axis=1)].shape[0]
display('Количество строк, в которых есть хотя бы одно пропущенное значение: {0}'.format(none_rows))

'Количество строк, в которых есть хотя бы одно пропущенное значение: 2174'

In [49]:
#пропущенная часть данных по столбцам
df.isna().mean()

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

In [50]:
days_employed_nan_ratio = df.isna().mean().days_employed
'Доля пропущенных значений в столбце days_employed: {:.0%}'.format(days_employed_nan_ratio)

'Доля пропущенных значений в столбце days_employed: 10%'

In [51]:
total_income_nan_ratio = df.isna().mean().total_income
'Доля пропущенных значений в столбце total_income: {:.0%}'.format(total_income_nan_ratio)

'Доля пропущенных значений в столбце total_income: 10%'

In [52]:
days_employed_median = df['days_employed'].abs().median().astype(int)
'Медианное значение days_employed: {0}'.format(days_employed_median)

'Медианное значение days_employed: 2194'

In [53]:
total_income_median = df['total_income'].abs().median().astype(int)
'Медианное значение total_income: {0}'.format(total_income_median)

'Медианное значение total_income: 145017'

In [54]:
#заполним пропуски медианой
df['days_employed'] = (
    df['days_employed']
    .fillna(df.groupby(by='income_type')['days_employed']
    .transform('median'))
)

df['total_income'] = (
    df['total_income']
    .fillna(df.groupby(by='income_type')['total_income']
    .transform('median'))
)

In [55]:
#проверим таблицу на пропуски
df[df.isna().any(axis=1)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          0 non-null      int64  
 1   days_employed     0 non-null      float64
 2   dob_years         0 non-null      int64  
 3   education         0 non-null      object 
 4   education_id      0 non-null      int64  
 5   family_status     0 non-null      object 
 6   family_status_id  0 non-null      int64  
 7   gender            0 non-null      object 
 8   income_type       0 non-null      object 
 9   debt              0 non-null      int64  
 10  total_income      0 non-null      float64
 11  purpose           0 non-null      object 
dtypes: float64(2), int64(5), object(5)
memory usage: 0.0+ bytes


### Проверка данных на аномалии и исправления.

In [56]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


В таблице встречаются отрицательные значения, которые должны быть положительными. И не приведенные к одному регистру слова.

In [57]:
#делаем значения положительными
df[['days_employed', 'total_income']] = df[['days_employed','total_income']].abs()
df['children'] = df['children'].abs()

In [58]:
#приводим слова к одному регистру
df[['education', 'family_status', 'income_type', 'purpose']] = df[
    ['education', 'family_status', 'income_type', 'purpose']].applymap(str.lower)

In [59]:
df.groupby(by='children')['children'].count()

children
0     14149
1      4865
2      2055
3       330
4        41
5         9
20       76
Name: children, dtype: int64

В столбце `children` ещё одна аномалия. 76 раз встречается число 20. 20 детей - крайне маловероятно. Скорее всего при вводе данных случайно был приписан 0.

In [60]:
#заменим 20 на 2
df.loc[df['children'] == 20, 'children'] = 2

### Изменение типов данных.

Два столбца содержат данные вещественного типа. Для дохода в рублях и стажа в днях это лишнее. Преобразуем в целочисленный.

In [61]:
df[['days_employed', 'total_income']] = df[['days_employed', 'total_income']].astype(int)

### Удаление дубликатов.

In [62]:
duplicates = df[df.duplicated()]['total_income'].count()
display('Дублирующихся строк: {0}'.format(duplicates))

'Дублирующихся строк: 71'

Сложно сказать, почему в таблице дублируются некоторые записи. Возможно заёмщик несколько раз подавал заявку на кредит. Или же есть сбой в сборе данных. В любом случае такие повторы нам не нужны - удалим.

In [63]:
df = df.drop_duplicates().reset_index(drop=True)
duplicates = df[df.duplicated()]['total_income'].count()
display('Дублирующихся строк: {0}'.format(duplicates))

'Дублирующихся строк: 0'

### Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Работать с таблицей удобней, когда категориальные данные в ней представлены коротким ключом, а "словарь" с парами ключ-значение хранится в отдельной таблице.

In [64]:
#создадим таблицу, содержающую перечень образовательных категорий.
education_dict = df[['education_id', 'education']].drop_duplicates().reset_index(drop=True)
education_dict

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [65]:
#создадим таблицу, содержающую перечень категорий семейного статуса.
family_status_dict = df[['family_status_id', 'family_status']].drop_duplicates().reset_index(drop=True)
family_status_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


In [66]:
#теперь удалим не нужные для анализа столбцы
df = df.drop(columns=['days_employed', 'dob_years', 'education', 'family_status'])
df.head()

,children,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,0,0,F,сотрудник,0,253875,покупка жилья
1,1,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,1,0,M,сотрудник,0,145885,покупка жилья
3,3,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,1,1,F,пенсионер,0,158616,сыграть свадьбу


### Категоризация дохода.

Для нашей задачи храненить количественные данные в столбце `total_income` - неудобно. Наглядней будет представить доход категориями.
- A - от 1млн₽
- B - от 200т₽ до 1млн₽
- С - от 50т₽ до 200т₽
- D - от 30т₽ до 50т₽
- E - до 30т₽

In [67]:
#функция вернет категорию в соответствии с диапазоном дохода в ячейке
def categorize_income(row):
    if 0 <=row <=30000:
        return 'E'
    elif 30001 <= row <= 50000:
        return 'D'
    elif 50001 <= row <= 200000:
        return 'C'
    elif 200001 <= row <= 1000000:
        return 'B'
    return 'A'

#добавим к таблице новый столбец с соответствующими категориями
try:
    df['total_income_id'] = df['total_income'].apply(categorize_income)
except:
    display('Проверьте значение столбца total_income')

df.head()

,children,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_id
0,1,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


### Категоризация целей кредита.

In [68]:
#отобразим уникальные цели кредита, встречающиеся в таблице
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

В столбце `purpose` цели кредита представлены ненаглядно. Если посмотреть внимательно, то все они сводятся к:
- образование
- недвижимость
- автомобиль
- свадьба

Категоризуем цели.

In [69]:
#функция найдёт в ячейке ключевые слова и вёрнет односложную цель.
def change_purpose(row):
    if ('жиль' in row) or ('недвижимо' in row):
        return 'недвижимость'
    elif 'недвижи' in row:
        return 'недвижимость'
    elif 'образовани' in row:
        return 'образование'
    elif 'свадьб' in row:
        return 'свадьба'
    elif 'авто' in row:
        return 'автомобиль'
    return 'другое'

#применим к столбцу purpose функцию change_purpose и перезапишем его
try:
    df['purpose'] = df['purpose'].apply(change_purpose)
except:
    display('Проверьте значения столбца purpose')

df.head()

,children,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_id
0,1,0,0,F,сотрудник,0,253875,недвижимость,B
1,1,1,0,F,сотрудник,0,112080,автомобиль,C
2,0,1,0,M,сотрудник,0,145885,недвижимость,C
3,3,1,0,M,сотрудник,0,267628,образование,B
4,0,1,1,F,пенсионер,0,158616,свадьба,C


## Ответы на вопросы:

Как связан риск невозврата кредита в срок с:
- количеством детей
- семейным положением
- доходом
- целью кредита

Для ответа на этот вопрос необходимо сгруппировать данные по интересующим нас категориям и посмотреть на процент невозврата в каждой.

In [70]:
#на вход функции подаётся название категории
#на выходе получим агрегированную по этой категории таблицу, где индекс - подкатегории, а столбцы -
#количество заёмщиков и среднее значение (или риск) в каждой. Сортируем в порядке убывания риска.
def calculate_risk(category, data=df):
    return data.pivot_table(
        index=category, values='debt', aggfunc={'mean', 'count'}).sort_values('mean', ascending=False)

#эту функцию будем использовать для объединения полученной таблицы рисков с таблицей "словаря"
#family_dict и education_dict
def merge(risk_table, dictionary, on):
    return dictionary.merge(risk_table, on=on, how='left').sort_values(
        by='mean', ascending=False).drop(columns=on)


In [71]:
#посчитаем интересующие риски и отобразим их в процентном формате.

family_risk = calculate_risk('family_status_id')
family_risk = merge(family_risk, family_status_dict, 'family_status_id').style.format({'mean': '{:,.1%}'.format}).hide_index()

education_risk = calculate_risk('education_id')
education_risk = merge(education_risk, education_dict, 'education_id').style.format({'mean': '{:,.1%}'.format}).hide_index()

total_income_risk = calculate_risk('total_income_id').style.format({'mean': '{:,.1%}'.format})

children_risk = calculate_risk('children').style.format({'mean': '{:,.1%}'.format})

purpose_risk = calculate_risk('purpose').style.format({'mean': '{:,.1%}'.format})

#### Вопрос 1: Дети

In [72]:
children_risk

,count,mean
children,,
4,41.000000,9.8%
2,2128.000000,9.5%
1,4855.000000,9.2%
3,330.000000,8.2%
0,14091.000000,7.5%
5,9.000000,0.0%


Бездетные заёмщики возвращают кредит в срок чаще остальных. Они же являются и крупнейшей категорией.  
В целом больше детей - больше риск. Однако многодетные ровно с тремя - исключение. Они подходят к возврату кредита ответственно.  
Статистика по заёмщикам с пятью детьми непоказательна, данных мало.

#### Вопрос 2: Семейное положение

In [73]:
family_risk

family_status,count,mean
не женат / не замужем,2810.000000,9.8%
гражданский брак,4151.000000,9.3%
женат / замужем,12339.000000,7.5%
в разводе,1195.000000,7.1%
вдовец / вдова,959.000000,6.6%


Успевшие побывать в браке - несут минимальный риск. Никогда не состоявшие в браке - максимальный.

#### Вопрос 3: Уровень дохода

In [74]:
total_income_risk

,count,mean
total_income_id,,
E,22.000000,9.1%
C,16015.000000,8.5%
A,25.000000,8.0%
B,5042.000000,7.1%
D,350.000000,6.0%


Заёмщики с доходом от 30т₽ до 50т₽ и от 200т₽ до 1млнт₽ - самые ответственные. Интересно, что если доход находится между 50т₽ и 200т₽ - риск повышается. По категориям до 30т₽ и больше 1млн₽ мало данных, чтобы оценивать риски.

#### Вопрос 4: Цели кредита

In [75]:
purpose_risk

,count,mean
purpose,,
автомобиль,4306.000000,9.4%
образование,4013.000000,9.2%
свадьба,2324.000000,8.0%
недвижимость,10811.000000,7.2%


Желающие приобрести недвижимость - представляют минимальные риски. И эта категория заёмщиков - самая большая.

#### Вопрос 5: Образование

In [76]:
education_risk

education,count,mean
начальное,282.000000,11.0%
неоконченное высшее,744.000000,9.1%
среднее,15172.000000,9.0%
высшее,5250.000000,5.3%
ученая степень,6.000000,0.0%


В целом, приятней иметь дело с более образованным человеком.   
Заёмщики с высшим образованием чаще других, причём аж на 3.7% минимум, отдают кредиты в срок. Есть предположение, что обладатели учёной степени - ещё более ответственны, хоть данных по ним крайне мало.  
Любопытно, что неоконченное высшее образование, пусть и всего на 0.1%, несёт банку больший риск, чем среднее.

## Общий вывод:

На основе данных о ~20тыс. заёмщиках:
- Меньший риск невозврата кредита в срок представляют  люди более образованные, состоявшие раньше в браке, бездетные и желающие приобрести недвижимость, с уровнем дохода от 30т₽ до 50т₽ или от 200т₽ до 1млн₽.
- Категории заёмщиков: ученая степень, доходы меньше 30т₽ и больше 1млн₽, многодетные - представлены слабо. Стоит изучить их глубже и, возможно, дать задачу маркетологам привлечь новых клиентов из этих категорий.
- Более образованные люди ответственней подходят к возврату кредита. В долгосрочной перспективе банку, возможно, выгодно будет разработать образовательные курсы и предлагать их заёмщикам (текущим и, может, потенциальным) на взаимовыгодных условиях.

В полученных данных были аномалии:
- Пропущенные значения в столбцах `total_income` и `days_employed`. Возможно это результат нежелания клиента раскрывать информацию, но может и ошибка сбора данных. Необходимо выяснить.
- Дубликаты строк. Возможно это повторные обращения клиента в банк, но зачем их сохранять при этом? Необходимо разобраться.
- Отрицательные значения в столбцах `days_employed`, `total_income`, `children`. А также аномальное несколько раз встречающееся `20` в столбце `children`. Узнать, как получаются такие значения.
- Столбцы, не приведенные к одному регистру, и столбец `purpose` с не приведёнными к единой форме категориями - мешают обработке данных. Возможно стоит обновить программу для их сбора.